# Data Load

In [1]:
import os
import pandas as pd
import numpy as np
import json
import pickle

In [2]:
data_path = "d:\\data\\store-sales-time-series-forecasting"
train_df = pd.read_csv(os.path.join(data_path, "train.csv"))
test_df = pd.read_csv(os.path.join(data_path, "test.csv"))
stores_df = pd.read_csv(os.path.join(data_path, "stores.csv"))
oil_df = pd.read_csv(os.path.join(data_path, "oil.csv"))
holidays_events_df = pd.read_csv(os.path.join(data_path, "holidays_events.csv"))
transactions_df = pd.read_csv(os.path.join(data_path, "transactions.csv"))

In [3]:
with open('null_to_mean.json', 'r') as f:
    null_to_mean = json.load(f)
    
dcoilwtice_mean = null_to_mean['dcoilwtice_mean']
log_transactions_mean = null_to_mean['log_transactions_mean']

In [4]:
with open('models/scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)
    
with open('models/date_oe.pkl', 'rb') as f:
    date_oe = pickle.load(f)
    
with open('models/category_oe.pkl', 'rb') as f:
    oe = pickle.load(f)
    
with open('models/pca_model.pkl', 'rb') as f:
    pca = pickle.load(f)

# Data Preprocessing

In [5]:
oil_df['dcoilwtico'].fillna(oil_df['dcoilwtico'].mean(), inplace=True)
test_df['log_onpromotion'] = np.log1p(test_df['onpromotion'])
test_df.drop(labels = ['onpromotion'], axis = 1, inplace = True)
stores_df['city'] = ["Other" if city != "Quito" and city != "Guayaquil" else city for city in stores_df['city']]
stores_df['state'] = ["Other" if state != "Pichincha" and state != "Guayas" else state for state in stores_df['state']]
locale_name_value = ['Ecuador', 'Quito', 'Riobamba', 'Guaranda', 'Latacunga', 'Ambato', 'Guayaquil']
holidays_events_df['locale_name'] = ["Other" if locale_name not in locale_name_value else locale_name for locale_name in holidays_events_df['locale_name']]
transactions_df['log_transactions'] = np.log1p(transactions_df['transactions'])
transactions_df.drop(labels = ['transactions'], axis = 1, inplace = True)
test_df.drop('id', axis = 1, inplace = True)

## Merge

In [6]:
merge_test_df = pd.merge(test_df, stores_df, how='left', on="store_nbr")
merge_test_df = pd.merge(merge_test_df, oil_df, how='left', on='date')
merge_test_df['dcoilwtico'].fillna(dcoilwtice_mean, inplace = True)
merge_test_df = pd.merge(merge_test_df, holidays_events_df, how='left', on='date')
merge_test_df.rename(columns = {"type_x":"store_type", "type_y":"holiday_type"}, inplace = True)
merge_test_df.fillna("None", inplace = True)
merge_test_df['transferred'] = merge_test_df['transferred'].astype("str")
merge_test_df = pd.merge(merge_test_df, transactions_df, how='left', on=['date', 'store_nbr'])
merge_test_df['log_transactions'].fillna(log_transactions_mean, inplace = True)
merge_test_df['date'] = merge_test_df['date'].astype('datetime64')
merge_test_df['year'] = merge_test_df['date'].dt.year
merge_test_df['month'] = merge_test_df['date'].dt.month
merge_test_df['day'] = merge_test_df['date'].dt.day
merge_test_df['weekday'] = merge_test_df['date'].dt.weekday
merge_test_df.drop(columns = 'date', inplace = True)

# Standard Scaler, One-Hot Encoding, PCA

In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA

In [8]:
# 수치형 컬럼
numeric_col = ['log_onpromotion', 'dcoilwtico', 'log_transactions']
# 카테고리 컬럼
category_col = ['store_nbr', 'city', 'state', 'store_type', 'cluster', 'holiday_type', 'locale', 'locale_name', 'transferred', 'family', 'description']
# 요일은 판매량에 큰 영향을 끼친다고 판단하여 pca에 따로 적용하지 않고 원핫인코딩만 함
weekday_col = ['month', 'weekday']

In [9]:
test_numeric_df = pd.DataFrame(scaler.transform(merge_test_df[numeric_col]), columns= scaler.get_feature_names_out())
test_wd_oe_df = pd.DataFrame(date_oe.transform(merge_test_df[weekday_col]), columns = date_oe.get_feature_names_out())
test_oe_df = pd.DataFrame(oe.transform(merge_test_df[category_col]), columns = oe.get_feature_names_out())

In [10]:
components = pca.components_
component_var_mapping = [f'PC{i+1}_{test_oe_df.columns[j]}' for i, j in enumerate(np.abs(components).argmax(axis=1))]
test_oe_select_df = pd.DataFrame(pca.transform(test_oe_df), columns = component_var_mapping)

In [11]:
merge_test_oe_df = pd.concat([test_numeric_df, test_wd_oe_df, test_oe_select_df], axis = 1)
merge_test_oe_df

,log_onpromotion,dcoilwtico,log_transactions,month_1,month_2,month_3,month_4,month_5,month_6,month_7,...,PC59_store_nbr_26,PC60_store_nbr_7,PC61_store_nbr_6,PC62_store_nbr_38,PC63_store_nbr_21,PC64_store_nbr_54,PC65_store_nbr_30,PC66_store_nbr_2,PC67_store_nbr_49,PC68_store_nbr_35
0,-0.336705,-1.097735,-0.007658,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.061389,0.172316,-0.275387,-0.011739,0.097562,-0.147920,0.000938,-0.183020,-0.301699,0.117077
1,-0.336705,-1.097735,-0.007658,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.076683,0.172316,-0.275387,-0.011739,0.097562,-0.147920,0.000938,-0.183020,-0.301699,0.117077
2,1.217904,-1.097735,-0.007658,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.070329,0.172316,-0.275387,-0.011739,0.097562,-0.147920,0.000938,-0.183020,-0.301699,0.117077
3,3.971496,-1.097735,-0.007658,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.063846,0.172316,-0.275387,-0.011739,0.097562,-0.147920,0.000938,-0.183020,-0.301699,0.117077
4,-0.336705,-1.097735,-0.007658,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.074606,0.172316,-0.275387,-0.011739,0.097562,-0.147920,0.000938,-0.183020,-0.301699,0.117077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28507,0.644144,-1.077038,-0.007658,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.107158,0.000076,-0.000183,-0.000183,-0.000290,-0.000718,-0.000353,0.001791,0.000967,-0.012860
28508,-0.336705,-1.077038,-0.007658,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.101067,0.000076,-0.000183,-0.000183,-0.000290,-0.000718,-0.000353,0.001791,0.000967,-0.012860
28509,0.644144,-1.077038,-0.007658,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.099046,0.000076,-0.000183,-0.000183,-0.000290,-0.000718,-0.000353,0.001791,0.000967,-0.012860
28510,2.921606,-1.077038,-0.007658,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.095917,0.000076,-0.000183,-0.000183,-0.000290,-0.000718,-0.000353,0.001791,0.000967,-0.012860


# Model Test

In [12]:
import torch
from models.LinearModel import LinearModel

In [13]:
x_test = torch.Tensor(merge_test_oe_df.values)

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LinearModel(input_features= merge_test_oe_df.shape[1], out_features= 1, hidden_dim_1= 128, hidden_dim_2= 64).to(device)
model.load_state_dict(torch.load('models/linear_model5.pt'))
model.eval()

LinearModel(
  (layer): Sequential(
    (0): Linear(in_features=90, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [15]:
y_test_pred = model(x_test)

In [16]:
test_df = pd.read_csv(os.path.join(data_path, "test.csv"))
y_test_pred = pd.DataFrame(y_test_pred.cpu().detach().numpy())
submission = pd.concat([test_df['id'], y_test_pred], axis = 1)
submission.rename(columns={0:"sales"}, inplace=True)
submission['sales'] = np.expm1(submission['sales'])     # Predicted by log value
submission

,id,sales
0,3000888,0.654921
1,3000889,24.229820
2,3000890,50.643707
3,3000891,71.849518
4,3000892,1.114651
...,...,...
28507,3029395,0.114332
28508,3029396,697.779358
28509,3029397,130.526535
28510,3029398,64.041496
